# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

Importando as bibliotecas a serem utilizadas:

In [65]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from emoji import UNICODE_EMOJI

Lendo a planilha de treinamento:

In [66]:
treinamento = pd.read_excel('tweets_xbox_treinamento.xlsx')

Limpando a base de dados:

In [67]:
bad_char = [':', '.', ';', '"', '?', '!', '{', '}', '*', '$', '[', ']', '(', ')', '/', '\n',
            '\t', '&', '<', '>', "'"]
space_char = ['\n', '\t', ',']
tweets = []
treinamento_parcial = pd.DataFrame()

for tweet in treinamento.Tweets:
    x = ''
    for i in range(len(tweet)):
        if tweet[i] in UNICODE_EMOJI:
            x = x + ' ' + tweet[i] + ' '
        elif tweet[i] in space_char:
            x = x + ' '
        elif tweet[i] not in bad_char:
            x = x + tweet[i]
    tweets.append(x)
treinamento_parcial['Tweets'] = tweets
treinamento_parcial['Classificação'] = treinamento['Classificação']

In [68]:
treinamento_limpo = pd.DataFrame()
tweets = []
tweet_final = ' '
for tweet in treinamento_parcial.Tweets:
    limpa_palavras = []
    lista_palavras = tweet.split(' ')
    for palavra in lista_palavras:
        if palavra in UNICODE_EMOJI:
            limpa_palavras.append(palavra)
        elif len(palavra) > 3 and palavra[:4] != 'http' and palavra[0] != '@' \
        and palavra[0] != '#' or palavra == 'ps4':
            limpa_palavras.append(palavra)
    tweets.append(tweet_final.join(limpa_palavras))
treinamento_limpo['Tweets'] = tweets
treinamento_limpo['Classificação'] = treinamento['Classificação']

Listando todas as palavras e contando o número de palavras que pertencem a tweets relevantes e não relevantes:

In [69]:
every_word = []

for i in range(len(treinamento_limpo['Tweets'])):
    lista_palavras = treinamento_limpo['Tweets'][i].split(' ')
    for palavra in lista_palavras:
        if palavra not in every_word:
            every_word.append(palavra)
            
words_relevante = 0
words_nao_relevante = 0

for i in range(len(treinamento_limpo['Tweets'])):
    lista_palavras = treinamento_limpo['Tweets'][i].split(' ')
    if treinamento_limpo['Classificação'][i] == 1:
        for palavra in lista_palavras:
            words_relevante += 1
    else:
        for palavra in lista_palavras:
            words_nao_relevante += 1

Dicionarios com a frequencia das palavras: (Laplace Smoothing já implementado)

In [70]:
relevante = {}
nao_relevante = {}

for word in every_word:
    relevante[word] = 1
    nao_relevante[word] = 1

for i in range(len(treinamento_limpo['Tweets'])):
    lista_palavras = treinamento_limpo['Tweets'][i].split(' ')
    if treinamento_limpo['Classificação'][i] == 1:
        for palavra in lista_palavras:
            if palavra in relevante:
                relevante[palavra] += 1
    else:
        for palavra in lista_palavras:
            if palavra in nao_relevante:
                nao_relevante[palavra] += 1

Calculando a P( palavra | relevante ) e P( palavra | não relevante ):

In [71]:
P_relevante = {}
P_nao_relevante = {}

for palavra in every_word:
    if palavra in relevante:
        P_relevante[palavra] = relevante[palavra] / (words_relevante + len(every_word))
    elif palavra in nao_relevante:
        P_nao_relevante[palavra] = nao_relevante[palavra] / (words_nao_relevante + len(every_word))

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
